In [98]:
import pandas as pd
import numpy as np

diabetes = pd.read_csv('data/diabetes_data.csv')

In [99]:
diabetes_un = diabetes.drop_duplicates()
print(diabetes.shape, diabetes_un.shape)

(778, 10) (768, 10)


In [100]:
def get_low_information_cols(data, porog=0.95):
    low_information_cols = []
    for col in data.columns:
        top_freq = data[col].value_counts(normalize=True).max()
        nunique_ratio = data[col].nunique() / data[col].count()
        if top_freq > porog:
            low_information_cols.append(col)
            # print(f'{col}: {round(top_freq*100,2)}% одинаковых значений')
        if nunique_ratio > porog:
            low_information_cols.append(col)
            # print(f'{col}: {round(nunique_ratio*100,2)}% уникальных значений')
    return low_information_cols

print(get_low_information_cols(diabetes_un))

['Gender']


In [101]:
diabetes_un2 = diabetes_un.drop('Gender', axis=1)
print(diabetes_un.shape, diabetes_un2.shape)

(768, 10) (768, 9)


In [102]:
print(diabetes_un2[diabetes_un2['Insulin'].isnull()].shape)

(0, 9)


In [111]:

def nan_func(x):
    return np.nan if x == 0 else x

diabetes_un3 = diabetes_un2.copy()
cols_0_to_NUN = ('Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI')

for col in cols_0_to_NUN:
    diabetes_un3[col] = diabetes_un3[col].apply(nan_func)

print(diabetes_un3[diabetes_un3['Insulin'].isnull()].shape, diabetes_un3.shape, round(diabetes_un3[diabetes_un3['Insulin'].isnull()].shape[0] / diabetes_un3.shape[0], 2))
print(diabetes_un3.isnull().mean().round(2).sort_values(ascending=False))

(0, 9) (768, 9) 0.0
Pregnancies                 0.0
Glucose                     0.0
BloodPressure               0.0
SkinThickness               0.0
Insulin                     0.0
BMI                         0.0
DiabetesPedigreeFunction    0.0
Age                         0.0
Outcome                     0.0
dtype: float64


In [109]:
thresh = diabetes_un3.shape[0] * 0.7
print(thresh)
diabetes_un4 = diabetes_un3.dropna(thresh=thresh, axis=1)
print(diabetes_un3.shape, diabetes_un4.shape)

537.5999999999999
(768, 9) (768, 0)


In [77]:
m = diabetes_un4.shape[1]
diabetes_un4 = diabetes_un4.dropna(thresh=m-2, axis=0)
print(diabetes_un4.shape)

(396, 10)
